In [1]:
# Import necessary modules and ignore file warnings
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Try to understand what the data is really about
df = pd.read_csv("mexico_ds/covid-19_general_MX.csv")
df.head()

,ID,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,...,INMUSUPR,HIPERTENSION,OTRA_CON,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,UCI
0,0,4,9,2,9,1,3/23/2020,3/22/2020,9999-99-99,97,...,2,2,2,2,2,2,2,99,1,97
1,1,4,9,2,9,2,4/13/2020,4/4/2020,9999-99-99,2,...,2,2,2,2,1,2,2,99,1,2
2,2,4,8,1,8,2,4/15/2020,4/10/2020,4/19/2020,2,...,2,1,2,2,1,2,2,99,1,2
3,3,4,30,1,30,1,4/27/2020,4/17/2020,9999-99-99,97,...,2,2,1,2,2,2,2,99,1,97
4,4,3,15,2,15,2,6/6/2020,6/1/2020,9999-99-99,2,...,2,2,2,2,2,2,2,1,1,2


In [3]:
# Start by dropping some unnecessary columns which don't add useful information
df.drop('ID', axis=1, inplace=True)
df.drop('SECTOR', axis=1, inplace=True)
df.drop('FECHA_INGRESO', axis=1, inplace=True)
df.drop('FECHA_SINTOMAS', axis=1, inplace=True)
df.drop('ENTIDAD_UM', axis=1, inplace=True)
df.drop('NACIONALIDAD', axis=1, inplace=True)
df.drop('ENTIDAD_RES', axis=1, inplace=True)
df.head()

,SEXO,TIPO_PACIENTE,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_CON,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,UCI
0,2,1,9999-99-99,97,2,28,2,2,2,2,2,2,2,2,2,2,99,1,97
1,2,2,9999-99-99,2,1,49,2,2,2,2,2,2,2,1,2,2,99,1,2
2,1,2,4/19/2020,2,1,67,2,2,2,2,1,2,2,1,2,2,99,1,2
3,1,1,9999-99-99,97,2,41,2,2,2,2,2,1,2,2,2,2,99,1,97
4,2,2,9999-99-99,2,1,43,2,2,2,2,2,2,2,2,2,2,1,1,2


In [4]:
# Translate the headings from Mexico to English
df.rename(columns={'SEXO':'sex','TIPO_PACIENTE':'attention',
                   'FECHA_DEF':'death','INTUBADO':'intubated','NEUMONIA':'pneumonia',
                   'EDAD':'age','DIABETES':'diabetes','EPOC':'epoch','ASMA':'asthma',
                   'INMUSUPR':'immunosuppression','HIPERTENSION':'hypertension',
                   'OTRA_CON':'other_diseases','CARDIOVASCULAR':'cardiovascular',
                   'OBESIDAD':'obesity',"RENAL_CRONICA":"renal_disease",'TABAQUISMO':'tabaquism',
                   'OTRO_CASO':'contact','RESULTADO':'result','UCI':'care'}, inplace=True)
df.head()

,sex,attention,death,intubated,pneumonia,age,diabetes,epoch,asthma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,renal_disease,tabaquism,contact,result,care
0,2,1,9999-99-99,97,2,28,2,2,2,2,2,2,2,2,2,2,99,1,97
1,2,2,9999-99-99,2,1,49,2,2,2,2,2,2,2,1,2,2,99,1,2
2,1,2,4/19/2020,2,1,67,2,2,2,2,1,2,2,1,2,2,99,1,2
3,1,1,9999-99-99,97,2,41,2,2,2,2,2,1,2,2,2,2,99,1,97
4,2,2,9999-99-99,2,1,43,2,2,2,2,2,2,2,2,2,2,1,1,2


In [5]:
# Represent Death as a boolean value instead of date
df.loc[df['death']=='9999-99-99','death']=0
df.loc[df['death']!=0,'death']=1
df.head()

,sex,attention,death,intubated,pneumonia,age,diabetes,epoch,asthma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,renal_disease,tabaquism,contact,result,care
0,2,1,0,97,2,28,2,2,2,2,2,2,2,2,2,2,99,1,97
1,2,2,0,2,1,49,2,2,2,2,2,2,2,1,2,2,99,1,2
2,1,2,1,2,1,67,2,2,2,2,1,2,2,1,2,2,99,1,2
3,1,1,0,97,2,41,2,2,2,2,2,1,2,2,2,2,99,1,97
4,2,2,0,2,1,43,2,2,2,2,2,2,2,2,2,2,1,1,2


In [6]:
# Represent age as classes. Numbers as such only add unneeded complexity to the data
# 0-20 -> Class 0
# 20-40 -> Class 1
# 40-60 -> Class 2
# 60--- -> Class 3
df.loc[(df['age']>=0) & (df['age']<20),'age']=0
df.loc[(df['age']>=20) & (df['age']<40),'age']=1
df.loc[(df['age']>=40) & (df['age']<60),'age']=2
df.loc[(df['age']>=60),'age']=3
df.head()

,sex,attention,death,intubated,pneumonia,age,diabetes,epoch,asthma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,renal_disease,tabaquism,contact,result,care
0,2,1,0,97,2,1,2,2,2,2,2,2,2,2,2,2,99,1,97
1,2,2,0,2,1,2,2,2,2,2,2,2,2,1,2,2,99,1,2
2,1,2,1,2,1,3,2,2,2,2,1,2,2,1,2,2,99,1,2
3,1,1,0,97,2,2,2,2,2,2,2,1,2,2,2,2,99,1,97
4,2,2,0,2,1,2,2,2,2,2,2,2,2,2,2,2,1,1,2


In [7]:
# Similarly replace all No's in the columns with 0 instead of default value of 2
df.loc[df['sex']==2,'sex']=0
df.loc[df['attention']==2,'attention']=0
df.loc[df['intubated']==2,'intubated']=0
df.loc[df['diabetes']==2,'diabetes']=0
df.loc[df['epoch']==2,'epoch']=0
df.loc[df['asthma']==2,'asthma']=0
df.loc[df['immunosuppression']==2,'immunosuppression']=0
df.loc[df['hypertension']==2,'hypertension']=0
df.loc[df['other_diseases']==2,'other_diseases']=0
df.loc[df['cardiovascular']==2,'cardiovascular']=0
df.loc[df['obesity']==2,'obesity']=0
df.loc[df['renal_disease']==2,'renal_disease']=0
df.loc[df['tabaquism']==2,'tabaquism']=0
df.loc[df['contact']==2,'contact']=0
df.loc[df['result']==2,'result']=0
df.loc[df['care']==2,'care']=0
df.head()

,sex,attention,death,intubated,pneumonia,age,diabetes,epoch,asthma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,renal_disease,tabaquism,contact,result,care
0,0,1,0,97,2,1,0,0,0,0,0,0,0,0,0,0,99,1,97
1,0,0,0,0,1,2,0,0,0,0,0,0,0,1,0,0,99,1,0
2,1,0,1,0,1,3,0,0,0,0,1,0,0,1,0,0,99,1,0
3,1,1,0,97,2,2,0,0,0,0,0,1,0,0,0,0,99,1,97
4,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,1,1,0


In [8]:
# Drop all rows where data is ignored (98) or unspecified (99) or no result (3)
df.drop(df[df.intubated==98].index, inplace=True)
df.drop(df[df.intubated==99].index, inplace=True)
df.drop(df[df.attention==98].index, inplace=True)
df.drop(df[df.attention==99].index, inplace=True)
df.drop(df[df.pneumonia==98].index, inplace=True)
df.drop(df[df.pneumonia==99].index, inplace=True)
df.drop(df[df.diabetes==98].index, inplace=True)
df.drop(df[df.diabetes==99].index, inplace=True)
df.drop(df[df.epoch==98].index, inplace=True)
df.drop(df[df.epoch==99].index, inplace=True)
df.drop(df[df.asthma==98].index, inplace=True)
df.drop(df[df.asthma==99].index, inplace=True)
df.drop(df[df.immunosuppression==98].index, inplace=True)
df.drop(df[df.immunosuppression==99].index, inplace=True)
df.drop(df[df.hypertension==98].index, inplace=True)
df.drop(df[df.hypertension==99].index, inplace=True)
df.drop(df[df.other_diseases==98].index, inplace=True)
df.drop(df[df.other_diseases==99].index, inplace=True)
df.drop(df[df.cardiovascular==98].index, inplace=True)
df.drop(df[df.cardiovascular==99].index, inplace=True)
df.drop(df[df.obesity==98].index, inplace=True)
df.drop(df[df.obesity==99].index, inplace=True)
df.drop(df[df.renal_disease==98].index, inplace=True)
df.drop(df[df.renal_disease==99].index, inplace=True)
df.drop(df[df.tabaquism==98].index, inplace=True)
df.drop(df[df.tabaquism==99].index, inplace=True)
df.drop(df[df.contact==98].index, inplace=True)
df.drop(df[df.contact==99].index, inplace=True)
df.drop(df[df.care==98].index, inplace=True)
df.drop(df[df.care==99].index, inplace=True)
df.drop(df[df.result==3].index, inplace=True)
df.head()

,sex,attention,death,intubated,pneumonia,age,diabetes,epoch,asthma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,renal_disease,tabaquism,contact,result,care
4,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,1,1,0
5,1,1,0,97,2,2,0,0,0,0,0,0,0,0,0,0,1,1,97
6,0,1,1,97,1,2,0,0,0,0,0,0,0,0,0,0,0,1,97
10,0,1,0,97,2,2,0,0,0,0,0,0,0,1,0,0,1,1,97
12,0,1,0,97,1,1,0,0,0,0,0,1,0,1,0,0,1,1,97


In [9]:
# Finally set 97 values to 2, as a third potential value of not applicable
df.loc[df['attention']==97,'attention']=2
df.loc[df['intubated']==97,'intubated']=2
df.loc[df['diabetes']==97,'diabetes']=2
df.loc[df['epoch']==97,'epoch']=2
df.loc[df['asthma']==97,'asthma']=2
df.loc[df['immunosuppression']==97,'immunosuppression']=2
df.loc[df['hypertension']==97,'hypertension']=2
df.loc[df['other_diseases']==97,'other_diseases']=2
df.loc[df['cardiovascular']==97,'cardiovascular']=2
df.loc[df['obesity']==97,'obesity']=2
df.loc[df['renal_disease']==97,'renal_disease']=2
df.loc[df['tabaquism']==97,'tabaquism']=2
df.loc[df['contact']==97,'contact']=2
df.loc[df['result']==97,'result']=2
df.loc[df['care']==97,'care']=2
df.head()

,sex,attention,death,intubated,pneumonia,age,diabetes,epoch,asthma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,renal_disease,tabaquism,contact,result,care
4,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,1,1,0
5,1,1,0,2,2,2,0,0,0,0,0,0,0,0,0,0,1,1,2
6,0,1,1,2,1,2,0,0,0,0,0,0,0,0,0,0,0,1,2
10,0,1,0,2,2,2,0,0,0,0,0,0,0,1,0,0,1,1,2
12,0,1,0,2,1,1,0,0,0,0,0,1,0,1,0,0,1,1,2


In [10]:
# Remove Duplicate Values
df.drop_duplicates(keep=False,inplace=True) 
df.head()

,sex,attention,death,intubated,pneumonia,age,diabetes,epoch,asthma,immunosuppression,hypertension,other_diseases,cardiovascular,obesity,renal_disease,tabaquism,contact,result,care
297,0,1,0,2,1,3,1,0,0,0,1,0,0,1,0,1,1,1,2
419,1,0,1,0,2,2,0,0,0,0,0,1,0,0,0,0,1,1,0
469,0,0,0,1,1,3,0,0,0,1,1,1,0,1,0,0,1,1,0
525,1,1,0,2,1,3,0,0,0,0,0,0,1,1,1,0,0,1,2
605,1,1,0,2,2,2,0,0,0,0,1,1,1,1,0,0,1,1,2


In [11]:
# List total records present in cleaned dataset
df.shape

(7891, 19)

In [12]:
# Save cleaned dataset
df.to_csv('export/mx_clean.csv',index=False)